In [91]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import MaxNLocator


In [92]:
libs = {}
libs['SRR1197125'] = 'germinating seed embryo'
#libs['SRR1197126'] = 'seedling leaf'
#libs['SRR1197127'] = 'seedling root'
#libs['SRR1197128'] = 'grain of 8 days after pollination'
path_transcripts = '/home/juan/Desktop/juan/bio/mirna_mite/data/fixed.cdna.all.fa'
path_annotation = '/home/juan/Desktop/juan/bio/mirna_mrcv/data/iwgsc_refseqv1.0_FunctionalAnnotation_v1__HCgenes_v1.0.TAB'
path_mirnas_deg = '/home/juan/Desktop/juan/bio/mirna_mrcv/data/mirna.deg.csv'
path_mirnas = '/home/juan/Desktop/juan/bio/mirna_mrcv/data/results.valid.csv'
path_libs = '/home/juan/Desktop/juan/bio/mirna_mrcv/data/sun_deg/%s'
path_transcripts_res = '/home/juan/Desktop/juan/bio/mirna_mrcv/data/targets.fa'


In [93]:
dfs = []
for lib,description in libs.items():
    path_lib = path_libs % lib
    df = pd.read_csv(path_lib, sep='\t',comment='#')
    df['lib'] = lib
    df['lib_description'] = description
    dfs.append(df)

In [94]:
df_cleave = pd.concat(dfs)
print(len(df_cleave.index))

376


In [95]:
df_cleave.drop_duplicates(['SiteID','Query'], inplace=True)
print(len(df_cleave.index))

376


In [96]:
df_cleave = df_cleave.reset_index(drop=True)

In [97]:
print(len(df_cleave[df_cleave.DegradomeCategory == 0].index))


15


In [98]:
df_cleave.head(2)

,SiteID,Query,Transcript,TStart,TStop,TSlice,MFEperfect,MFEsite,MFEratio,AllenScore,Paired,Unpaired,Structure,Sequence,DegradomeCategory,DegradomePval,Tplot_file_path,lib,lib_description
0,TraesCS1A02G004200.1:338,Cluster_5906 2A:485771022-485771154 l2fc:-0.91...,TraesCS1A02G004200.1,325,347,338,-40.7,-27.1,0.665848,12.5,"3-11,345-337;12-18,333-327","1-2,347-346[UP5];x-x,336-334[BULt];19-20,326-3...",..(((((((...(((((((((..&..)))))))))---)))))))..,AAGGCCAAGUAUGUGACCGUCGU&UAGGCGGUCAC---CUUGGCUAG,4,1.0,/home/juan/Desktop/juan/bio/mirna_mrcv/data/su...,SRR1197125,germinating seed embryo
1,TraesCS1A02G142500.1:1345,Cluster_30793 6D:95858420-95858697 l2fc:0.581922,TraesCS1A02G142500.1,1333,1354,1345,-43.0,-28.3,0.658140,8.0,"2-11,1353-1344;13-20,1341-1334","1-1,1354-1354[UP5];12-12,1343-1342[AILt];21-21...",.((((((((..((((((((((.&.))))))))))-.)))))))).,ACUUGAACAGACACGGAGCUGA&GCGGCUCUGUG-GUGUUCAAGC,2,1.0,/home/juan/Desktop/juan/bio/mirna_mrcv/data/su...,SRR1197125,germinating seed embryo


In [99]:
df_cleave = df_cleave[['Query','Transcript','DegradomeCategory','DegradomePval','TStart','TStop','TSlice','lib','lib_description']]


In [100]:
df_cleave['Query'] = df_cleave['Query'].str.split(' ').str[0]

In [101]:
df_cleave.head(2)

,Query,Transcript,DegradomeCategory,DegradomePval,TStart,TStop,TSlice,lib,lib_description
0,Cluster_5906,TraesCS1A02G004200.1,4,1.0,325,347,338,SRR1197125,germinating seed embryo
1,Cluster_30793,TraesCS1A02G142500.1,2,1.0,1333,1354,1345,SRR1197125,germinating seed embryo


In [102]:
df_mirs_degs = pd.read_csv(path_mirnas_deg, sep=',')

In [103]:
df_mirs_degs.head(2)

,Gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,X21dpiR1C.trimmed,X21dpiR3C.trimmed,X21dpiR1T.trimmed,X21dpiR3T.trimmed
0,Cluster_9889,298.598043,-1.471187,0.264059,-5.571425,2.526645e-08,0.000002,417.884204,459.336142,137.863421,179.308403
1,Cluster_35781,411.843895,-0.908261,0.236510,-3.840258,1.229050e-04,0.005899,583.212178,491.732645,306.089619,266.341137


In [104]:
df_mirs_degs = df_mirs_degs[['Gene','log2FoldChange']]

In [105]:
df_mirs_degs.head(2)

,Gene,log2FoldChange
0,Cluster_9889,-1.471187
1,Cluster_35781,-0.908261


In [106]:
df_mirs = pd.read_csv(path_mirnas, sep='\t')
df_mirs.head(2)

,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,DicerCall,MIRNA,PhaseScore,Short,Long,20,21,22,23,24
0,1A:309399489-309399664,Cluster_872,176,553,4.340,1,0.002,-,ACGGCAUAGAGGCACUGCAAA,327,...,21,Y,-1.0,7,1,35,440,61,5,4
1,1A:419025472-419025570,Cluster_1015,99,384,3.014,20,1.000,+,CGAAUGUAUUUUUUAUGGCUUG,249,...,22,Y,-1.0,2,3,27,83,264,3,2


In [108]:
df_annotation = pd.read_csv(path_annotation, sep='\t')
print(len(df_annotation.index))
df_annotation.head(2)

137052


,Gene-ID,is_repr,AHRD-Quality-Code,Blast-Hit-Accession,Human-Readable-Description,Pfam-IDs-(Description),Interpro-IDs-(Description),GO-IDs-(Description)-via-Interpro,Gene-or-TE-TE?-U (via-function),Pfam-IDs,Interpro-IDs,GO-IDs-via-Interpro
0,TraesCS1A01G000100.1,1,-**,sp|P0AGJ7|TRML_ECOLI,tRNA (cytidine(34)-2'-O)-methyltransferase,NaN,NaN,NaN,G,NaN,NaN,NaN
1,TraesCS1A01G000200.1,1,--*,sp|Q9NS15|LTBP3_HUMAN,Latent-transforming growth factor beta-binding...,NaN,NaN,NaN,G,NaN,NaN,NaN


In [109]:
df_annotation = df_annotation[['Gene-ID','Human-Readable-Description']]

In [81]:
#merge df_cleave df_mirs df_mirs_degs df_annotation

In [89]:
df_1 = pd.merge(df_mirs, df_mirs_degs, left_on='Name', right_on='Gene')
print(len(df_1.index))
df_1.drop(['Gene'], axis=1, inplace=True)
df_1.head(3)

11


,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,MIRNA,PhaseScore,Short,Long,20,21,22,23,24,log2FoldChange
0,2A:485771022-485771154,Cluster_5906,133,5985,46.972,305,1.000,+,UAGGCGGUCACCUUGGCUAG,2883,...,Y,-1.0,1080,3,4050,798,52,0,2,-0.915816
1,2A:652328872-652329011,Cluster_6309,140,1185,9.300,1,0.001,-,UGCCAAAGGAGAGUUGCCCUG,1136,...,Y,-1.0,14,0,3,1155,11,2,0,-1.201318
2,2A:707747480-707747579,Cluster_6516,100,1271,9.975,1,0.999,+,UGGACGAGGAUGUGCAGCUGC,988,...,Y,-1.0,9,2,2,1256,2,0,0,0.908011


In [90]:
df_2 = pd.merge(df_1, df_cleave, left_on='Name', right_on='Query')
print(len(df_2.index))
df_2.drop(['Query'], axis=1, inplace=True)
df_2.head(10)

376


,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,24,log2FoldChange,Transcript,DegradomeCategory,DegradomePval,TStart,TStop,TSlice,lib,lib_description
0,2A:485771022-485771154,Cluster_5906,133,5985,46.972,305,1.0,+,UAGGCGGUCACCUUGGCUAG,2883,...,2,-0.915816,TraesCS1A02G004200.1,4,1.000000,325,347,338,SRR1197125,germinating seed embryo
1,2A:485771022-485771154,Cluster_5906,133,5985,46.972,305,1.0,+,UAGGCGGUCACCUUGGCUAG,2883,...,2,-0.915816,TraesCS1A02G308100.1,3,0.999774,2015,2034,2025,SRR1197125,germinating seed embryo
2,2A:485771022-485771154,Cluster_5906,133,5985,46.972,305,1.0,+,UAGGCGGUCACCUUGGCUAG,2883,...,2,-0.915816,TraesCS1A02G308100.2,4,1.000000,2018,2037,2028,SRR1197125,germinating seed embryo
3,2A:485771022-485771154,Cluster_5906,133,5985,46.972,305,1.0,+,UAGGCGGUCACCUUGGCUAG,2883,...,2,-0.915816,TraesCS1A02G350200.1,4,1.000000,1026,1049,1037,SRR1197125,germinating seed embryo
4,2A:485771022-485771154,Cluster_5906,133,5985,46.972,305,1.0,+,UAGGCGGUCACCUUGGCUAG,2883,...,2,-0.915816,TraesCS1A02G350200.4,4,1.000000,855,878,866,SRR1197125,germinating seed embryo
5,2A:485771022-485771154,Cluster_5906,133,5985,46.972,305,1.0,+,UAGGCGGUCACCUUGGCUAG,2883,...,2,-0.915816,TraesCS1B02G005600.1,3,1.000000,1168,1190,1181,SRR1197125,germinating seed embryo
6,2A:485771022-485771154,Cluster_5906,133,5985,46.972,305,1.0,+,UAGGCGGUCACCUUGGCUAG,2883,...,2,-0.915816,TraesCS1B02G018900.1,2,1.000000,658,677,668,SRR1197125,germinating seed embryo
7,2A:485771022-485771154,Cluster_5906,133,5985,46.972,305,1.0,+,UAGGCGGUCACCUUGGCUAG,2883,...,2,-0.915816,TraesCS1B02G202700.1,4,1.000000,1407,1427,1418,SRR1197125,germinating seed embryo
8,2A:485771022-485771154,Cluster_5906,133,5985,46.972,305,1.0,+,UAGGCGGUCACCUUGGCUAG,2883,...,2,-0.915816,TraesCS1B02G364500.3,3,0.999990,1025,1048,1036,SRR1197125,germinating seed embryo
9,2A:485771022-485771154,Cluster_5906,133,5985,46.972,305,1.0,+,UAGGCGGUCACCUUGGCUAG,2883,...,2,-0.915816,TraesCS1B02G364500.4,3,0.999982,956,979,967,SRR1197125,germinating seed embryo


In [ ]:
df_3 = pd.merge(df_2, df_annotation, left_on='Transcript', right_on='Transcript')
print(len(df_3.index))
df_3.drop(['Transcript_y'], axis=1, inplace=True)
df_3.head(10)